In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 14657499
paper_name = 'willingham_muchowski_2003' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [21]:
files = ['alpha-sinuclein.txt','huntingtin.txt']

In [23]:
original_data_list = []

for f in files:
    original_data = pd.read_csv('raw_data/' + f, sep='\t')
    print('Original data dimensions: %d x %d' % (original_data.shape))
    print(original_data.head())
    original_data['gene'] = original_data['Strain'].apply(lambda x: x.split(' ')[1])
    
    # Eliminate all white spaces & capitalize
    original_data['gene'] = clean_genename(original_data['gene'])
    
    typo_fixes = {'MRP11':'MRPL1','PC16':'PCL6'}
    original_data['gene'] = original_data['gene'].apply(lambda x: typo_fixes[x] if x in typo_fixes.keys() else x)
    
    # Translate to ORFs  
    original_data['orf'] = translate_sc(original_data['gene'], to='orf')
    
    # Make sure everything translated ok
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    
    original_data['data'] = -1
    
    original_data.set_index('orf', inplace=True)
    original_data = original_data[['data']].copy()
    
    original_data = original_data.groupby(original_data.index).mean()
    print(original_data.shape)
    
    original_data_list.append(original_data)

Original data dimensions: 86 x 3
     Strain Ortholog                                      Function
0  1. ape2Δ      Yes                                Aminopeptidase
1  2. arl3Δ      Yes           ARF small monomeric GTPase activity
2  3. aro1Δ       No                   Arom pentafunctional enzyme
3  4. cog6Δ      Yes  Involved in vesicular transport to the Golgi
4  5. crh1Δ      Yes                             Cell wall protein
Empty DataFrame
Columns: [Strain, Ortholog, Function, gene, orf]
Index: []
(86, 1)
Original data dimensions: 52 x 3
     Strain Ortholog                                          Function
0  1. apj1Δ      Yes                                   Hsp40 chaperone
1  2. apm2Δ      Yes                    Nonselective vesicle transport
2  3. ayr1Δ      Yes  Ketoreductase; acylglycerone-phosphate reductase
3  4. cit2Δ      Yes                     Citrate synthase, peroxisomal
4  5. cmk1Δ      Yes                          Protein histidine kinase
Empty DataFrame
Columns

In [24]:
original_data = pd.concat(original_data_list, axis=1)

In [26]:
original_data.index.name='orf'

In [27]:
original_data[original_data.isnull()] = 0

# Prepare the final dataset

In [28]:
data = original_data.copy()

In [29]:
dataset_ids = [202, 70]
datasets = datasets.reindex(index=dataset_ids)

In [30]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [31]:
data.head()

dataset_id,202,70
data_type,value,value
orf,,
YAL022C,-1.0,0.0
YAR035W,-1.0,0.0
YBR013C,-1.0,0.0
YBR284W,-1.0,0.0
YBR295W,-1.0,0.0


## Subset to the genes currently in SGD

In [32]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [33]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,202,70
,data_type,value,value
gene_id,orf,,
20,YAL022C,-1.0,0.0
77,YAR035W,-1.0,0.0
209,YBR013C,-1.0,0.0
478,YBR284W,-1.0,0.0
489,YBR295W,-1.0,0.0


# Normalize

In [34]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [35]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [36]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,202,70,202,70
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
20,YAL022C,-1.0,0.0,-7.288092,0.0
77,YAR035W,-1.0,0.0,-7.288092,0.0
209,YBR013C,-1.0,0.0,-7.288092,0.0
478,YBR284W,-1.0,0.0,-7.288092,0.0
489,YBR295W,-1.0,0.0,-7.288092,0.0


# Print out

In [37]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [38]:
from IO.save_data_to_db3 import *

In [39]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 14657499...
Inserting the new data...


100%|██████████| 2/2 [00:00<00:00,  3.01it/s]

Updating the data_modified_on field...
